In [66]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, Tokenizer
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import  IDF
from pyspark.ml import Pipeline
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import desc
from pyspark.ml.clustering import KMeans
from pyspark.ml.clustering import LDA
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.classification import LogisticRegression
import pyspark.sql.functions as f
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import lit
from pyspark.ml.linalg import Vectors, VectorUDT

In [2]:
%time
spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("Text_Classification") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

spark.sparkContext.setLogLevel('WARN')

print(spark.version)

Wall time: 0 ns


2.3.2


In [3]:
path = '/Users/User/PycharmProjects/spark/animals_comments.csv'
data = spark.read.csv([path])
print("Number of documents read in is:", data.count())

Number of documents read in is: 5820036


In [4]:
%%time
data = data.withColumnRenamed('_c0', 'chan_owner').withColumnRenamed('_c1', 'id_comentator').withColumnRenamed('_c2', 'text')
data.show(5, truncate = False)

+-------------+-------------+-----------------------------------------------------------------------------------------------------------------------+
|chan_owner   |id_comentator|text                                                                                                                   |
+-------------+-------------+-----------------------------------------------------------------------------------------------------------------------+
|creator_name |userid       |comment                                                                                                                |
|Doug The Pug | 87          |I shared this to my friends and mom the were lol                                                                       |
|Doug The Pug | 87          |Super cute  😀🐕🐶                                                                                                     |
|bulletproof  | 530         |stop saying get em youre literally dumb . have some common sense or dont o

In [5]:
data.groupBy("id_comentator") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(n=50, truncate=40)

+-------------+-----+
|id_comentator|count|
+-------------+-----+
|      2036522| 2571|
|       455571| 2159|
|       569313| 1452|
|      1727704| 1339|
|      2041593| 1288|
|      2288680| 1247|
|       954873|  888|
|       479268|  775|
|      1568280|  757|
|       575285|  742|
|      2184324|  740|
|       170012|  725|
|       797741|  697|
|      1367232|  671|
|      1293328|  644|
|      2056517|  642|
|      1705977|  627|
|      2071628|  605|
|      1448539|  601|
|       717385|  596|
|      2427846|  596|
|      2396540|  586|
|       576447|  581|
|       239250|  578|
|      1766767|  573|
|      1194456|  566|
|         null|  565|
|      2399121|  563|
|       416516|  558|
|      2042389|  544|
|      1810444|  541|
|      2427873|  541|
|       620667|  538|
|      2207685|  523|
|      1377368|  517|
|      2076074|  512|
|      1288301|  509|
|       635762|  508|
|      2451022|  503|
|      1007044|  498|
|      1078410|  487|
|       248498|  487|
|       20

In [19]:
#regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
#wordsDataFrame = regexTokenizer.transform(data)
#wordsDataFrame.show()

In [7]:
countDistinctDF = data.select("id_comentator", "text")\
  .groupBy("id_comentator", "text").agg(countDistinct("text"))

countDistinctDF.head(3)

[Row(id_comentator=' 100009', text='I love my zoo!!!!', count(DISTINCT text)=1),
 Row(id_comentator=' 1000130', text='why i am here ?And i just like the video....i donno why....', count(DISTINCT text)=1),
 Row(id_comentator=' 1000536', text='Taylor make love commet pls', count(DISTINCT text)=1)]

In [23]:
countDistinctDF.describe().show()


+-------+------------------+--------------------+--------------------+
|summary|     id_comentator|                text|count(DISTINCT text)|
+-------+------------------+--------------------+--------------------+
|  count|           5747660|             5747191|             5747661|
|   mean|1269911.4608634575|            Infinity|  0.9999182276059775|
| stddev| 733793.1145652495|                 NaN|0.009042440020482887|
|    min|                 1|                    |                   0|
|    max|            userid|🧡🧡💛🧡💛🧡💛🧡?...|                   1|
+-------+------------------+--------------------+--------------------+



In [ ]:
countDistinctDF.withColumn('word', f.explode(f.split(f.col('text'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .show(20)

+-------+------------------+--------------------+--------------------+
|summary|     id_comentator|                text|count(DISTINCT text)|
+-------+------------------+--------------------+--------------------+
|  count|           5747660|             5747191|             5747661|
|   mean|1269911.4608634575|            Infinity|  0.9999182276059775|
| stddev| 733793.1145652495|                 NaN|0.009042440020482887|
|    min|                 1|                    |                   0|
|    max|            userid|🧡🧡💛🧡💛🧡💛🧡?...|                   1|
+-------+------------------+--------------------+--------------------+



In [34]:
words_list_cat = ['cat owner', 'cats owner', "кот", 'i have cat', 'my cat', 'cat', 'cats']

cat = countDistinctDF.filter(f.col('text')
                            .rlike('(^|\s)(' + '|'
                            .join(words_list_cat) + ')(\s|$)'))

cat.describe().show()

+-------+------------------+--------------------+--------------------+
|summary|     id_comentator|                text|count(DISTINCT text)|
+-------+------------------+--------------------+--------------------+
|  count|            124016|              124016|              124016|
|   mean|1264328.2589101407|                null|                 1.0|
| stddev| 730971.5050201144|                null|                 0.0|
|    min|           1000034| . . .Okay thats ...|                   1|
|    max|            999990|🤣🤣Lmao when the...|                   1|
+-------+------------------+--------------------+--------------------+



In [32]:
words_list_dog = ['dog', 'dogs', "собака", 'i have dog', 'dog owner', "my dog", "mine dog"]
#i am dog
dog = countDistinctDF.filter(f.col('text')
                            .rlike('(^|\s)(' + '|'
                            .join(words_list_dog) + ')(\s|$)'))

dog.describe().show()

+-------+-----------------+--------------------+--------------------+
|summary|    id_comentator|                text|count(DISTINCT text)|
+-------+-----------------+--------------------+--------------------+
|  count|           192114|              192114|              192114|
|   mean|1270143.804912708|                null|                 1.0|
| stddev|732612.2757294886|                null|                 0.0|
|    min|          1000014| A dog walking on...|                   1|
|    max|           999976|🤣🤣🤣🤣🤣my dog ...|                   1|
+-------+-----------------+--------------------+--------------------+



In [38]:
dog = dog.withColumn('category', lit("dog"))
cat = cat.withColumn('category', lit("cat"))


In [39]:
cat_dog = dog.unionAll(cat)


+-------------+--------------------+--------------------+--------+
|id_comentator|                text|count(DISTINCT text)|category|
+-------------+--------------------+--------------------+--------+
|       949820|stealing your dog...|                   1|     dog|
|      1459644|I did not see him...|                   1|     dog|
|      1614293|magnificent I can...|                   1|     dog|
|       208425|   amazing smart dog|                   1|     dog|
|      2374837|this is so sad se...|                   1|     dog|
|       532964|those were his wo...|                   1|     dog|
|      1248369|peoples we should...|                   1|     dog|
|      1843986|I am dog trainer ...|                   1|     dog|
|      1868536|What a lolfox. Th...|                   1|     dog|
|      2263155|I think this is t...|                   1|     dog|
|       437587|Now thats a true ...|                   1|     dog|
|       547114|    Cute dogs 😍😍😍|                   1|     dog

In [40]:
cat_dog.show(25)

+-------------+--------------------+--------------------+--------+
|id_comentator|                text|count(DISTINCT text)|category|
+-------------+--------------------+--------------------+--------+
|       949820|stealing your dog...|                   1|     dog|
|      1459644|I did not see him...|                   1|     dog|
|      1614293|magnificent I can...|                   1|     dog|
|       208425|   amazing smart dog|                   1|     dog|
|      2374837|this is so sad se...|                   1|     dog|
|       532964|those were his wo...|                   1|     dog|
|      1248369|peoples we should...|                   1|     dog|
|      1843986|I am dog trainer ...|                   1|     dog|
|      1868536|What a lolfox. Th...|                   1|     dog|
|      2263155|I think this is t...|                   1|     dog|
|       437587|Now thats a true ...|                   1|     dog|
|       547114|    Cute dogs 😍😍😍|                   1|     dog

In [41]:
cat_dog.drop("count(DISTINCT text)")

DataFrame[id_comentator: string, text: string, category: string]

In [ ]:
# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
# stop words
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")
# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)
label_stringIdx = StringIndexer(inputCol = "category", outputCol = "label")
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(cat_dog)
dataset = pipelineFit.transform(cat_dog)
dataset.show(5)


In [43]:
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 221493


Test Dataset Count: 94637


In [44]:
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)

predictions.filter(predictions['prediction'] == 0) \
    .select("id_comentator","text","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+-------------+------------------------------+--------+------------------------------+-----+----------+
|id_comentator|                          text|category|                   probability|label|prediction|
+-------------+------------------------------+--------+------------------------------+-----+----------+
|       237626|Here is the problem with pr...|     dog|[0.9999999999590736,4.09264...|  0.0|       0.0|
|      2017407|Place human-dog friendship ...|     dog|[0.9999999997818161,2.18184...|  0.0|       0.0|
|      2017407|Place human-dog friendship ...|     dog|[0.9999999982427308,1.75726...|  0.0|       0.0|
|      2017407|Bella is beautiful - she ne...|     dog|[0.9999999978576615,2.14233...|  0.0|       0.0|
|       706859|Hey Ian! Lovely to see you ...|     dog|[0.9999999892109029,1.07890...|  0.0|       0.0|
|      1831625|*WARNING: Long comment! I j...|     cat|[0.9999999071835253,9.28164...|  1.0|       0.0|
|      1686874|Has anyone actually read pi...|     dog|[0.999999

In [45]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.9378528948313585

In [62]:
import matplotlib.pyplot as plt
import numpy as np

trainingSummary = lrModel.summary
roc = trainingSummary.roc.toPandas()

In [63]:
plt.plot(roc['FPR'], roc['TPR'])
plt.ylabel('False Positive Rate')
plt.xlabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()
print('Training set areaUnderROC: ' + str(trainingSummary.areaUnderROC))


In [ ]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(smoothing=1)
model_nb = nb.fit(trainingData)
predictions = model_nb.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("id_comentator","text","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)


In [52]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.9449365850744954

In [56]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)

# Train model with Training Data
rfModel = rf.fit(trainingData)

In [57]:
predictions_rf = rfModel.transform(testData)
predictions_rf.filter(predictions_rf['prediction'] == 0) \
    .select("id_comentator","text","category","probability","label","prediction")\
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+-------------+------------------------------+--------+------------------------------+-----+----------+
|id_comentator|                          text|category|                   probability|label|prediction|
+-------------+------------------------------+--------+------------------------------+-----+----------+
|         8460|A bullmastiff is a form of ...|     dog|[0.7439669895522577,0.25603...|  0.0|       0.0|
|      2230282|I have a beautiful male Rid...|     dog|[0.7245044172289337,0.27549...|  0.0|       0.0|
|       602073|You know I like how this vi...|     dog|[0.7234606942531768,0.27653...|  0.0|       0.0|
|      1599417|All of these breeds are alr...|     dog|[0.7231087793344182,0.27689...|  0.0|       0.0|
|      2487142|dam that mutherfucker bit t...|     dog|[0.7220139497321232,0.27798...|  0.0|       0.0|
|      1599417|Awesome video.  This is wit...|     dog|[0.7206074195630413,0.27939...|  0.0|       0.0|
|      2230282|Not a bad list but I must s...|     dog|[0.719923

In [59]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions_rf)

0.4599925919799822

In [69]:
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
wordsDataFrame = regexTokenizer.transform(countDistinctDF)

Py4JJavaError: An error occurred while calling o1610.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 323.0 failed 1 times, most recent failure: Lost task 3.0 in stage 323.0 (TID 36482, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$2: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.agg_doAggregateWithKeysOutput_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:614)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:

In [70]:
wordsDataFrame.show()

Py4JJavaError: An error occurred while calling o1610.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 323.0 failed 1 times, most recent failure: Lost task 3.0 in stage 323.0 (TID 36482, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$2: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.agg_doAggregateWithKeysOutput_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:614)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:

In [ ]:
cv_tmp = CountVectorizer(inputCol="words", outputCol="tmp_vectors")
cv_tmp_model = cv_tmp.fit(wordsDataFrame)


Py4JJavaError: An error occurred while calling o1610.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 323.0 failed 1 times, most recent failure: Lost task 3.0 in stage 323.0 (TID 36482, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$2: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.agg_doAggregateWithKeysOutput_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:614)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala: